In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.feature import Imputer
import pyspark.sql.functions as F
from delta import *

In [ ]:
builder = SparkSession.builder \
    .appName("Reviews to Silver") \
    .config("spark.jars.packages", "io.delta:delta-core_2.12:3.1.0") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

In [ ]:
def load_reviews_data(spark, path):
    schema = StructType([
        StructField("review_id", StringType(), True),
        StructField("user_id", StringType(), True),
        StructField("business_id", StringType(), True),
        StructField("stars", DoubleType(), True),
        StructField("useful", IntegerType(), True),
        StructField("funny", IntegerType(), True),
        StructField("cool", IntegerType(), True),
        StructField("text", StringType(), True),
        StructField("date", StringType(), True)
    ])
    return spark.read.json(path, schema=schema)

df = load_reviews_data(spark, "D:/Project/delta_lake/bronze/yelp_academic_dataset_review.json")
df.show(5)

In [ ]:
def handle_missing_values(df):
    imputer = Imputer(
        inputCols=["stars", "useful", "funny", "cool"],
        outputCols=["stars_imputed", "useful_imputed", "funny_imputed", "cool_imputed"]
    ).setStrategy("median")
    
    df = imputer.fit(df).transform(df)
    
    df = df.na.fill({
        "text": "No review text provided",
        "date": "1970-01-01"
    })
    return df

df = handle_missing_values(df)
df.show(5)

In [ ]:
def standardize_data(df):
    return df \
        .withColumn("text", trim(lower(col("text")))) \
        .withColumn("date", to_timestamp(col("date"))) \
        .withColumn("year", year(col("date"))) \
        .withColumn("month", month(col("date"))) \
        .withColumn("day", dayofmonth(col("date")))

df = standardize_data(df)
df.show(5)

In [ ]:
def feature_engineering(df):
    df = df.withColumn("text_length", length(col("text")))
    
    df = df.withColumn("total_votes", 
        col("useful") + col("funny") + col("cool"))
    
    df = df.withColumn("rating_category",
        when(col("stars") >= 4.5, "Excellent")
        .when(col("stars") >= 4.0, "Very Good")
        .when(col("stars") >= 3.5, "Good")
        .when(col("stars") >= 3.0, "Average")
        .otherwise("Below Average"))
    
    df = df.withColumn("review_age_days", 
        datediff(current_date(), col("date")))
    
    return df

df = feature_engineering(df)
df.show(5)

In [ ]:
def validate_data(df):
    df = df.filter(
        (col("stars").between(1, 5)) &
        (col("useful") >= 0) &
        (col("funny") >= 0) &
        (col("cool") >= 0) &
        (length(col("text")) > 0)
    )
    
    df = df.filter(
        (length(col("review_id")) > 0) &
        (length(col("business_id")) > 0) &
        (length(col("user_id")) > 0)
    )
    
    return df

df = validate_data(df)
df.show(5)

In [ ]:
def quality_checks(df):
    null_counts = df.select([sum(col(c).isNull().cast("int")).alias(c) 
                           for c in df.columns])
    
    duplicate_count = df.count() - df.dropDuplicates().count()
    
    value_dist = df.select(
        mean("stars").alias("mean_stars"),
        stddev("stars").alias("stddev_stars"),
        mean("useful").alias("mean_useful"),
        mean("funny").alias("mean_funny"),
        mean("cool").alias("mean_cool")
    )
    
    print("Null Counts:")
    null_counts.show()
    print(f"\nDuplicate Count: {duplicate_count}")
    print("\nValue Distributions:")
    value_dist.show()
    
    return df

df = quality_checks(df)

In [ ]:
df.write.format("delta") \
    .option("mergeSchema", "true") \
    .mode("overwrite") \
    .partitionBy("year", "month") \
    .save("D:/Project/delta_lake/silver/reviews")